## The aim of this lab is to build a system for sentiment analysis on a dataset of tweets.

In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/jdariasl/DiplomadoEPM_2019/master/Modulo4/init.py
from init import init; init(force_download=False)

In [ ]:
import sys
if 'google.colab' in sys.modules:
    print ("setting tensorflow version in colab")
    %tensorflow_version 2.x
    %load_ext tensorboard
import tensorflow as tf
tf.__version__

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
data = pd.read_csv('local/data/Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [3]:
data

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [5]:
import re
#Suprimir la clase Neutral
data = data[data.sentiment != "Neutral"]

#Normalización de texto
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

4472
16986


In [6]:
data

,text,sentiment
1,rt scottwalker didnt catch the full gopdebate ...,Positive
3,rt robgeorge that carly fiorina is trending h...,Positive
4,rt danscavino gopdebate w realdonaldtrump deli...,Positive
5,rt gregabbott_tx tedcruz on my first day i wil...,Positive
6,rt warriorwoman91 i liked her and was happy wh...,Negative
...,...,...
13866,rt cappy_yarbrough love to see men who will ne...,Negative
13867,rt georgehenryw who thought huckabee exceeded ...,Positive
13868,rt lrihendry tedcruz as president i will alway...,Positive
13869,rt jrehling gopdebate donald trump says that h...,Negative


In [7]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

In [8]:
data

,text,sentiment
1,scottwalker didnt catch the full gopdebate l...,Positive
3,robgeorge that carly fiorina is trending ho...,Positive
4,danscavino gopdebate w realdonaldtrump deliv...,Positive
5,gregabbott_tx tedcruz on my first day i will...,Positive
6,warriorwoman91 i liked her and was happy whe...,Negative
...,...,...
13866,cappy_yarbrough love to see men who will nev...,Negative
13867,georgehenryw who thought huckabee exceeded t...,Positive
13868,lrihendry tedcruz as president i will always...,Positive
13869,jrehling gopdebate donald trump says that he...,Negative


In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

In [10]:
len(X)

10729

## Padding

Las secuencias no son de igual longitud entonces deben ser ajustadas para poder ser usadas como entrada a una red neuronal

In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X2 = pad_sequences(X, padding='post')
print(X2[0])

[ 363  122    1  703    2   39   58  237   37  210    6  174 1761   12
 1324 1409  743    0    0    0    0    0    0    0    0    0    0    0]


In [18]:
X = pad_sequences(X)
X[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        363,  122,    1,  703,    2,   39,   58,  237,   37,  210,    6,
        174, 1761,   12, 1324, 1409,  743], dtype=int32)

### Se debe acompañar de una capa de enmascaramiento [Masking](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Masking) o si la primera capa es un Embedding se debe activar la opción: mask_zero

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1],mask_zero=True))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 28, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


Note también el uso de la capa SpatialDropout1D

In [22]:
from sklearn.model_selection import train_test_split
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


In [23]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Train on 7188 samples
Epoch 1/7
7188/7188 - 8s - loss: 0.4250 - accuracy: 0.8208
Epoch 2/7
7188/7188 - 6s - loss: 0.3175 - accuracy: 0.8669
Epoch 3/7
7188/7188 - 6s - loss: 0.2770 - accuracy: 0.8894
Epoch 4/7
7188/7188 - 6s - loss: 0.2505 - accuracy: 0.9003
Epoch 5/7
7188/7188 - 7s - loss: 0.2268 - accuracy: 0.9076
Epoch 6/7
7188/7188 - 6s - loss: 0.2090 - accuracy: 0.9200
Epoch 7/7
7188/7188 - 6s - loss: 0.1903 - accuracy: 0.9265


In [24]:
validation_size = 1500
X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

2041/2041 - 1s - loss: 0.4411 - accuracy: 0.8295
score: 0.44
acc: 0.83


In [26]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 0)[0]
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
        
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 60.517799352750814 %
neg_acc 90.51217464315701 %


Validemos un tuit

In [30]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#twt = ['It was a really good experience']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 0)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  206  633    6  150    5   55 1055   55   46    6  150]]
negative


## También se puede construir usando redes convolucionales!